In [1]:
from networkx.readwrite import json_graph
import pickle, json
import networkx as nx
from tqdm import tqdm
import os

In [8]:
'''保存graph和各种list'''
workpwd = os.getcwd()
save_path = os.path.join(workpwd, "data")

with open(os.path.join(save_path, "G.pkl"), 'rb') as f:
    G = pickle.load(f)
with open(os.path.join(save_path, 'graphs_per_year.pkl'), 'rb') as f:
    graphs_per_year = pickle.load(f)

with open(os.path.join(save_path, 'name_list.pkl'), 'rb') as f:
    name_list = pickle.load(f)
with open(os.path.join(save_path, 'name_dict_list.pkl'), 'rb') as f:
    name_dict_list = pickle.load(f)
with open(os.path.join(save_path, 'name_dict_id_list.pkl'), 'rb') as f:
    name_dict_id_list = pickle.load(f)
with open(os.path.join(save_path, 'affiliation_set.pkl'), 'rb') as f:
    affiliation_set = pickle.load(f)
with open(os.path.join(save_path, 'uid_nameId_list.pkl'), 'rb') as f:
    uid_nameId_list = pickle.load(f)
with open(os.path.join(save_path, 'affiliation_dict.pkl'), 'rb') as f:
    affiliation_dict = pickle.load(f)
with open(os.path.join(save_path, 'affiliation_list.pkl'), 'rb') as f:
    affiliation_list = pickle.load(f)
with open(os.path.join(save_path, 'affiliation_id_list.pkl'), 'rb') as f:
    affiliation_id_list = pickle.load(f)

'''读取MetaData'''
MetaData_path = os.path.join(workpwd, "..", "database", "MetaData.json")
with open(MetaData_path, 'r', encoding='utf-8') as f:
    MetaData = json.load(f)

In [3]:
# G['affiliation']['北京大学']

In [9]:
class Addable_Graph(nx.Graph):
    def __add__(self, other):
        # Implement the addition logic here
        res = self.copy()
        for node in other.nodes:
            if node not in res.nodes:
                res.add_node(node)
            else:
                # res.nodes[node]['affiliation_list'] = name_dict_list[index]
                res.nodes[node]['paper_num'] += other.nodes[node]['paper_num']
                res.nodes[node]['citation'] += other.nodes[node]['citation']
                res.nodes[node]['years'].update(res.nodes[node]['years'])
        for edge in other.edges:
            if edge not in res.edges:
                res.add_edge(edge[0], edge[1], papers=other.edges[edge]['papers'], paper_num_weight=other.edges[edge]['paper_num_weight'], citation_weight=other.edges[edge]['citation_weight'])
            else:
                res[edge[0]][edge[1]]['papers'].update(other[edge[0]][edge[1]]['papers'])
                res[edge[0]][edge[1]]['paper_num_weight'] += other[edge[0]][edge[1]]['paper_num_weight']
                res[edge[0]][edge[1]]['citation_weight'] += other[edge[0]][edge[1]]['citation_weight']
        return res

    def __iadd__(self, other):
        # Implement the addition logic here
        for node in other.nodes:
            if node not in self.nodes:
                self.add_node(node)
            else:
                # res.nodes[node]['affiliation_list'] = name_dict_list[index]
                self.nodes[node]['paper_num'] += other.nodes[node]['paper_num']
                self.nodes[node]['citation'] += other.nodes[node]['citation']
                self.nodes[node]['years'].update(self.nodes[node]['years'])
        for edge in other.edges:
            if edge not in self.edges:
                self.add_edge(edge[0], edge[1], papers=other.edges[edge]['papers'], paper_num_weight=other.edges[edge]['paper_num_weight'], citation_weight=other.edges[edge]['citation_weight'])
            else:
                self[edge[0]][edge[1]]['papers'].update(other[edge[0]][edge[1]]['papers'])
                self[edge[0]][edge[1]]['paper_num_weight'] += other[edge[0]][edge[1]]['paper_num_weight']
                self[edge[0]][edge[1]]['citation_weight'] += other[edge[0]][edge[1]]['citation_weight']
        return self

class Sum_Graph(Addable_Graph):
    def __init__(self, G=None, graphs_per_year=None):
        if graphs_per_year and G:
            super().__init__(G)
            self.graphs_per_year = [Addable_Graph(graph) for graph in graphs_per_year]
            self.y0 = 1980
        else:
            pass

    def year(self, year0, year1=None):
        '''单个年份返回的是引用，多个年份返回的是新建的图'''
        res = self.graphs_per_year[year0-self.y0]
        if year1 and year1 > year0:
            res = res.copy()
            for year in range(year0+1, year1+1):
                res += self.graphs_per_year[year-self.y0]
            return res
        return res

    def affiliation(self, affiliation):
        return self.subgraph(affiliation_id_list[affiliation_dict[affiliation]])

In [10]:
graph_G = Sum_Graph(G, graphs_per_year)

In [13]:
graph_G.year(2001,2002).edges()

EdgeView([(8985, 8986), (8985, 8987), (8985, 8988), (8986, 8987), (8986, 8988), (8987, 8988), (10166, 10167), (43946, 43947), (54333, 54334), (54333, 54335), (54333, 54336), (54333, 54337), (54334, 54335), (54334, 54336), (54334, 54337), (54335, 54336), (54335, 54337), (54336, 54337), (59050, 59051), (59050, 59052), (59051, 59052), (73198, 73199), (73198, 73200), (73198, 73201), (73199, 73200), (73199, 73201), (73200, 73201), (78919, 78920), (78919, 78921), (78919, 78922), (78920, 78921), (78920, 78922), (78921, 78922), (82926, 82927), (82926, 82928), (82927, 82928), (43680, 43682), (43680, 43681), (43680, 491395), (43680, 287774), (43682, 43681), (43682, 491395), (115306, 115307), (115306, 115308), (115307, 115308), (119152, 151722), (119152, 224165), (119152, 40261), (119152, 21127), (119152, 82151), (119152, 245115), (123769, 123770), (123769, 123771), (123769, 123772), (123769, 123773), (123770, 123771), (123770, 123772), (123770, 123773), (123770, 37143), (123770, 169129), (123770

In [15]:
graph_G.year(2001,2002)[8985][8986], graph_G[8985][8986]

({'papers': {'015ae6a5cc767b28e087343dc6ea26968f54635c895937731e263ed8cf9d67cf': {'year': 2001,
    'citation': 13,
    'name_id_list': [8985, 8986, 8987, 8988],
    'name_id_len': 4}},
  'paper_num_weight': 1,
  'citation_weight': 13},
 {'papers': {'015ae6a5cc767b28e087343dc6ea26968f54635c895937731e263ed8cf9d67cf': {'year': 2001,
    'citation': 13,
    'name_id_list': [8985, 8986, 8987, 8988],
    'name_id_len': 4}},
  'paper_num_weight': 1,
  'citation_weight': 13})

In [12]:
for edge in graph_G.affiliation('北京大学').edges:
    print(edge, graph_G.edges[edge])
    print(MetaData[1][graph_G.edges[edge]['papers'].keys().__iter__().__next__()])

(96899, 96894) {'papers': {'22dc57dd01c862063b314f8ee04e50b5d5d483ceeccd26a11add682343191199': {'year': 2014, 'citation': 10, 'name_id_list': [96894, 80740, 96895, 96896, 96897, 96898, 38396, 22348, 96899], 'name_id_len': 9}}, 'paper_num_weight': 1, 'citation_weight': 10}
{'title': '中医辨证治疗慢性前列腺炎的疗效评价研究', 'authors': ['高国庆1', '宋竖旗2', '卢建新2', '孙自学3', '高文喜4', '顾晓箭5', '贺菊乔6', '李海松7', '张亚强1'], 'year': 2014, 'province': ['江苏', '河南', '湖北', '湖南', '四川', '北京'], 'affiliations': ['北京大学', '中国中医科学院广安门医院', '河南省中医院', '湖北省中医院', '江苏省中医院', '湖南中医药大学第一附属医院', '北京中医药大学东直门医院'], 'journal': '中国中医基础医学杂志', 'keywords': ['中医药', '辨证论治', '慢性前列腺炎', '疗效评价'], 'source': '北京大学；中国中医科学院广安门医院；河南省中医院；湖北省中医院；江苏省中医院；湖南中医药大学第一附属医院；北京中医药大学东直门医院  2014  第20卷  第10期  1394-1396，1401    ', 'abstract': '目的:客观评价中医辨证论治慢性前列腺炎(Chronic Prostatitis,CP)的临床疗效。方法:多中心、大样本、分层区组随机、西药对照临床研究设计,216例CP患者按1∶1比例随机分为治疗组(中医辨证组)和对照组(西医治疗组)各108例,疗程为4周,第6周随访。以美国国立卫生院前列腺炎症状评分(NIH-CPSI)、中医证候评分作为主要疗效评价指标。结果:1治疗4周后,治疗组和对照组NIH-CPSI评分分别为(12.2±6.04和15.1±7.57),治疗组低于对照